In [56]:
import xarray as xr
import pandas as pd
import numpy as np

In [48]:
era_old = pd.read_csv("data/ERA/era.csv")
era_old.head()

,time,lat,lon,rain,tmean,month,year
0,1993-01-01,-25.0,-69.0,0.000698,9.41,1,1993
1,1993-01-01,-25.0,-68.9,0.001047,9.21,1,1993
2,1993-01-01,-25.0,-68.8,0.001443,8.79,1,1993
3,1993-01-01,-25.0,-68.7,0.001846,8.63,1,1993
4,1993-01-01,-25.0,-68.6,0.002307,8.11,1,1993


In [49]:
tmean = xr.open_dataset("data/ERA/adaptor.mars.internal-1688106777.5563195-3400-10-ab04a334-090b-4b03-8618-e44f1bc55d69.grib").to_dataframe().reset_index().dropna().reset_index(drop=True)
rain1 = xr.open_dataset("data/ERA/adaptor.mars.internal-1688106991.2341475-30970-8-3470c5a2-6688-448c-aa7c-83cf8ce08a45.grib", filter_by_keys={'stepType': 'avgad'}).to_dataframe().reset_index()
rain2 = xr.open_dataset("data/ERA/adaptor.mars.internal-1688106991.2341475-30970-8-3470c5a2-6688-448c-aa7c-83cf8ce08a45.grib", filter_by_keys={'stepType': 'avgas'}).to_dataframe().reset_index()

rain = pd.concat([rain2, rain1], ignore_index=True).dropna().reset_index(drop=True)
era = tmean.merge(rain, on=["time", "latitude", "longitude", "number", "step", "surface", "valid_time"])

In [50]:
# Kelvin to Celsius
era["t2m"] = era["t2m"].apply(lambda x: x - 273.15)
# time to datetime
era["time"] = pd.to_datetime(era["time"])
era["valid_time"] = pd.to_datetime(era["valid_time"])

# Sort values
era = era.sort_values(by=["time", "latitude", "longitude"]).loc[:, ["time", "latitude", "longitude", "t2m", "tp"]].reset_index(drop=True)

era.columns = ["time", "lat", "lon", "tmean", "rain"]
era["month"] = era["time"].dt.month
era["year"] = era["time"].dt.year
era = era[era_old.columns]

In [54]:
era_old.head()

,time,lat,lon,rain,tmean,month,year
0,1993-01-01,-25.0,-69.0,0.000698,9.41,1,1993
1,1993-01-01,-25.0,-68.9,0.001047,9.21,1,1993
2,1993-01-01,-25.0,-68.8,0.001443,8.79,1,1993
3,1993-01-01,-25.0,-68.7,0.001846,8.63,1,1993
4,1993-01-01,-25.0,-68.6,0.002307,8.11,1,1993


In [63]:
era_complete = pd.concat([era, era_old], ignore_index=True).sort_values(by=["year", "month"]).reset_index(drop=True)

In [66]:
era_complete.to_csv("data/ERA/era_complete.csv", index=False)